In [20]:
import pandas as pd
import numpy as np

df = pd.read_excel("supplement_tablesv2.xlsx", sheet_name="TableS1MHC")

#temp = df[["allele", "peptide", "label_quantitative", "label", "PubMed ID", "NetMHCIIpan3.2", "PUFFIN", "MHCnuggets"]]
#temp.to_csv("data.csv",index=False)
df.head()

,allele,Alpha,Betha,mhc,peptide,PubMed ID,Method/Technique,Object Type,Assay Group,label,label_quantitative,BERTMHC-affinity,NetMHCIIpan3.2,PUFFIN,MHCnuggets
0,HLA-DRA1*01:01-DRB5*01:01,HLA-DRA1*01:01,DRB5*01:01,MAISGVPVLGFFIIAVLMSAQESWAIKEEHVIIQAEFYLNPDQSGE...,QTKEFQVLKSLGKLA,28871256.0,purified MHC/competitive/fluorescence,Linear peptide,half maximal inhibitory concentration (IC50),1.0,0.594998,0.982612,0.785,0.843472,0.783692
1,HLA-DRA1*01:01-DRB1*01:01,HLA-DRA1*01:01,DRB1*01:01,MAISGVPVLGFFIIAVLMSAQESWAIKEEHVIIQAEFYLNPDQSGE...,GASFYHLGAAVPHSQ,28751567.0,purified MHC/competitive/fluorescence,Linear peptide,half maximal inhibitory concentration (IC50),1.0,0.682411,0.950999,0.791,0.820125,0.903182
2,HLA-DRA1*01:01-DRB1*01:01,HLA-DRA1*01:01,DRB1*01:01,MAISGVPVLGFFIIAVLMSAQESWAIKEEHVIIQAEFYLNPDQSGE...,RTYSLGSALRPSTSR,28481943.0,purified MHC/competitive/radioactivity,Linear peptide,half maximal inhibitory concentration (IC50),1.0,0.778379,0.945474,0.722,0.711652,0.708052
3,HLA-DRA1*01:01-DRB1*01:01,HLA-DRA1*01:01,DRB1*01:01,MAISGVPVLGFFIIAVLMSAQESWAIKEEHVIIQAEFYLNPDQSGE...,RWGFRSGVPPKVVNY,28575582.0,purified MHC/competitive/fluorescence,Linear peptide,half maximal inhibitory concentration (IC50),0.0,0.307237,0.936212,0.789,0.813055,0.866972
4,HLA-DRA1*01:01-DRB1*01:01,HLA-DRA1*01:01,DRB1*01:01,MAISGVPVLGFFIIAVLMSAQESWAIKEEHVIIQAEFYLNPDQSGE...,EGFLKAQASLRGQYW,28751567.0,purified MHC/competitive/fluorescence,Linear peptide,half maximal inhibitory concentration (IC50),1.0,0.878086,0.935469,0.784,0.784606,0.756262


In [21]:
# solo extraemos las columnas que necesitamos para BERTMHC
data = age_sex = df[["mhc", "peptide", "label_quantitative", "label"]]
data.columns = ["mhc", "peptide", "label", "class"]
data.head()

,mhc,peptide,label,class
0,MAISGVPVLGFFIIAVLMSAQESWAIKEEHVIIQAEFYLNPDQSGE...,QTKEFQVLKSLGKLA,0.594998,1.0
1,MAISGVPVLGFFIIAVLMSAQESWAIKEEHVIIQAEFYLNPDQSGE...,GASFYHLGAAVPHSQ,0.682411,1.0
2,MAISGVPVLGFFIIAVLMSAQESWAIKEEHVIIQAEFYLNPDQSGE...,RTYSLGSALRPSTSR,0.778379,1.0
3,MAISGVPVLGFFIIAVLMSAQESWAIKEEHVIIQAEFYLNPDQSGE...,RWGFRSGVPPKVVNY,0.307237,0.0
4,MAISGVPVLGFFIIAVLMSAQESWAIKEEHVIIQAEFYLNPDQSGE...,EGFLKAQASLRGQYW,0.878086,1.0


In [22]:
# separamos por clase, chocoleamos y dividiremos en train, eval y test
class_1 = data.loc[data['class'] == 1]
class_0 = data.loc[data['class'] == 0]

from sklearn.utils import shuffle
from math import ceil, floor

class_1 = shuffle(class_1)
class_0 = shuffle(class_0)

print("number of class 1: ", class_1.shape[0])
print("number of class 0: ", class_0.shape[0])

eval_ratio = 0.15 # 15% para evaluación
test_ratio = 0.15 # 15% para testing
train_ratio = 0.7 # 70% para training

# train, eval,test, spit class 1 ###########################################
train_n = floor(train_ratio*class_1.shape[0])
eval_n = floor(eval_ratio*class_1.shape[0])
test_n = class_1.shape[0] - train_n - eval_n

train_class_1 = class_1[0: train_n]
eval_class_1 = class_1[train_n: train_n + eval_n]
test_class_1 = class_1[train_n + eval_n: -1]

print("train class 1:", train_class_1.shape[0])
print("eval class 1:", eval_class_1.shape[0])
print("test class 1:", test_class_1.shape[0])

# train, eval,test, spit class 0 ###########################################
train_n = floor(train_ratio*class_0.shape[0])
eval_n = floor(eval_ratio*class_0.shape[0])
test_n = class_0.shape[0] - train_n - eval_n

train_class_0 = class_0[0: train_n]
eval_class_0 = class_0[train_n: train_n + eval_n]
test_class_0 = class_0[train_n + eval_n: -1]

print("train class 0:", train_class_0.shape[0])
print("eval class 0:", eval_class_0.shape[0])
print("test class 0:", test_class_0.shape[0])

# merge all dataset
train = shuffle(pd.concat([train_class_1, train_class_0]))
eval = shuffle(pd.concat([eval_class_1, eval_class_0]))
test = shuffle(pd.concat([test_class_1, test_class_0]))

print("\nfinish train eval test split...")
print(train.shape)
print(eval.shape)
print(test.shape)



train.to_csv("train.csv",index=False)
eval.to_csv("eval.csv",index=False)
test.to_csv("test.csv",index=False)

number of class 1:  466
number of class 0:  1868
train class 1: 326
eval class 1: 69
test class 1: 70
train class 0: 1307
eval class 0: 280
test class 0: 280

finish train eval test split...
(1633, 4)
(349, 4)
(350, 4)
